In [66]:
import os
import sys
sys.path.append(os.path.abspath(os.pardir))
import pandas as pd
import json
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from config import Settings

In [14]:
s = Settings()

In [15]:
# loads in 129_000 recent transactions on base
df1 = pd.read_csv('transactions.csv')
df2 = pd.read_csv('transactions_1.csv')
df3 = pd.read_csv('transactions_2.csv')
df4 = pd.read_csv('transactions_3.csv')
df = pd.concat([df1, df2, df3, df4], ignore_index=True)
df.drop(columns=['Column 1', 'Column 5', 'Column 8', 'Column 12'], inplace=True)
df = df.rename(columns={
    'Column 2': 'Txn Hash',
    'Column 3': 'Method',
    'Column 4': 'Block',
    'Column 6': 'Age',
    'Column 7': 'From',
    'Column 9': 'To',
    'Column 10': 'Value',
    'Column 11': 'Txn Fee',
})

In [68]:
df.shape

(129000, 8)

In [67]:
# Displays the count of each unique value in the 'To' or 'From' column
pd.set_option('display.max_rows', None)
count = df['To'].value_counts()
print(df.shape)
print(count[:500])
# filtered_df = df[df['From'] == '0x9c71226863d3db3a7de3402e3743fea8026dc9e0']
# count = filtered_df['To'].value_counts()
# print(count)

In [18]:
# finds teh contracts that have the keywords in the 'To' column
keywords = ['swap', 'router', 'exchange']
pattern = '|'.join(keywords)

df_to_values = df[df['To'].str.contains(pattern, case=False, na=False)]['To']

# Make the list unique
unique_to_values = list(set(df_to_values))

for value in unique_to_values:
    print(value)

Uniswap V3: Staker 1.0.2
Uniswap V3: Nonfungible Position Manager
Aerodrome: Universal Router
KyberSwap: Double Signatures Limit Order
SwapBased: BASE Token
DackieSwap : V2 Router
Izumi Finance: swap
Squid: Router
DackieSwap : Non fungible Position Manager
AlienBase: Router
BSX Exchange: Contract
Chainlink: CCIP Router
ReHold: Router
Zerion: Router
PancakeSwap: Universal Router
DackieSwap: DACKIE Token
Uniswap: V2 Router02
Odos:Router V2
UNCX Lockers: UniSwap V3 LP
Paraswap v5: Augustus Swapper
Bitget Wallet: Swap Router v1
Vela Exchange : Order Vault
Synapse: Router
Create: SolarSwap
Banana Gun: Router
SwapBased : Uniswap V2 Router 02
PancakeSwap V3: Smart Router
Uniswap: Universal Router V1 2 V2Support
KyberSwap: Meta Aggregation Router v2
1inch: Aggregation Router V6
Uniswap: Universal Router V1 2
0x: Exchange Proxy
SwapBased : xBASE Token
Aerodrome: Router
Synthswap: MasterChef
Uniswap: Universal Router
Overnight Finance : Exchange
OpenOcean: Exchange V2
BaseSwap: BSWAP Token
Nethe

In [19]:
# Web scraping to find contract addresses

# List of contract names
contract_names = unique_to_values  # unique contract names from the previous step

# Initialize the WebDriver (e.g., using Chrome)
driver = webdriver.Chrome()

# Function to search for a contract and extract the address
def search_contract_and_get_address(contract_name):
    # Open the website
    driver.get("https://basescan.org/")
    
    # Wait for the search bar to load
    time.sleep(3)
    
    # Find the search bar and input the contract name
    search_bar = driver.find_element(By.ID, "txtSearchInput")
    search_bar.clear()
    search_bar.send_keys(contract_name)
    search_bar.send_keys(Keys.RETURN)
    
    # Wait for the results page to load
    time.sleep(5)
    
    try:
        # Extract the contract address
        address_element = driver.find_element(By.ID, "mainaddress")
        contract_address = address_element.text
    except:
        # If address is not found, return None
        contract_address = None
    
    return contract_address

# Dictionary to store contract names and addresses
contract_data = {}

# Iterate through the contract names and get their addresses
for name in contract_names:
    address = search_contract_and_get_address(name)
    if address:
        contract_data[name] = address
    else:
        contract_data[name] = "Address not found"
    time.sleep(2)  # Add delay to avoid overloading the server

# Close the WebDriver
driver.quit()

# Save the results to a JSON file
with open("contract_addresses.json", "w") as json_file:
    json.dump(contract_data, json_file, indent=4)

print("Contract addresses have been saved to contract_addresses.json")


Contract addresses have been saved to contract_addresses.json


# Obtaining ABIs

In [37]:
with open('contract_addresses.json') as f:
    data = json.load(f)

In [27]:
# Function to get the key from a value in a dictionary. Used to get the name of the contract from the address.
def get_key(val):
    for key, value in data.items():
        if val in value:
            return key
    return "key doesn't exist"

print(get_key(data['Bitget Wallet: Swap Router v1']))

Bitget Wallet: Swap Router v1


In [48]:
# Function to split the input string by a delimiter and replace spaces with underscores
def split_and_replace(s, delimiter=':'):
    parts = s.split(delimiter)
    if len(parts) != 2:
        raise ValueError("Input string should contain exactly one delimiter")
    folder = parts[0].strip().replace(' ', '_')
    file = parts[1].strip().replace(' ', '_')
    return folder, file

In [58]:
# creates folder and writes abi to file
def write_json_to_file(folder, filename, data):
    # Ensure the folder exists
    os.makedirs(folder, exist_ok=True)

    # Create the full file path
    file_path = os.path.join(folder, filename)

    # Write the data to the file
    with open(file_path + ".json", 'w') as f:
        json.dump(data, f)

In [64]:
# Calls basescan api to grab abi's
def get_abi(address):
    url = f"""https://api.basescan.org/api?module=contract&action=getabi&address={address}&apikey={s.base_scan_api_key}"""
    response = requests.get(url)
    if response.status_code == 200:
        r = response.json()
        abi_string = r['result']
        try:
            abi = json.loads(abi_string)
            return abi
        except json.JSONDecodeError:
            return None
    else:
        return None

In [65]:
# run the functions
for address in data.values():
    print(split_and_replace(get_key(address)))
    abi = get_abi(address)
    if abi:
        folder, file = split_and_replace(get_key(address))
        write_json_to_file(folder, file, abi)
    time.sleep(1)

('Uniswap_V3', 'Staker_1.0.2')
('Aerodrome', 'Universal_Router')
('SwapBased', 'BASE_Token')
('DackieSwap', 'V2_Router')
('Izumi_Finance', 'swap')
('Squid', 'Router')
('DackieSwap', 'Non_fungible_Position_Manager')
('AlienBase', 'Router')
('BSX_Exchange', 'Contract')
('Chainlink', 'CCIP_Router')
('ReHold', 'Router')
('Zerion', 'Router')
('PancakeSwap', 'Universal_Router')
('DackieSwap', 'DACKIE_Token')
('Uniswap', 'V2_Router02')
('Odos', 'Router_V2')
('UNCX_Lockers', 'UniSwap_V3_LP')
('Paraswap_v5', 'Augustus_Swapper')
('Bitget_Wallet', 'Swap_Router_v1')
('Vela_Exchange', 'Order_Vault')
('Synapse', 'Router')
('Banana_Gun', 'Router')
('SwapBased', 'Uniswap_V2_Router_02')
('PancakeSwap_V3', 'Smart_Router')
('Uniswap', 'Universal_Router_V1_2_V2Support')
('KyberSwap', 'Meta_Aggregation_Router_v2')
('1inch', 'Aggregation_Router_V6')
('Uniswap', 'Universal_Router_V1_2')
('0x', 'Exchange_Proxy')
('SwapBased', 'xBASE_Token')
('Aerodrome', 'Router')
('Synthswap', 'MasterChef')
('Uniswap', 'Univ